#### Handling Imbalanced Dataset with Machine Learning

In [1]:
import pandas as pd
df=pd.read_csv('ais_disabling_events_main_only_imp_col_csv.csv')
df.head()

,vessel_class,gap_hours,iuu_caught,spherical_distances,eez_check,Ocean List New whose false were in OG
0,other,13.016667,0,207.942845,0,atlantic
1,other,13.850000,0,9.704232,0,atlantic
2,other,33.733333,0,10.789069,1,atlantic
3,other,30.650000,0,11.695860,1,atlantic
4,other,50.283333,0,93.113335,1,atlantic


In [2]:
df.shape

(55368, 6)

In [3]:
df['iuu_caught'].value_counts()

0    55268
1      100
Name: iuu_caught, dtype: int64

In [4]:
#### Independent and Dependent Features
X=df.drop("iuu_caught",axis=1)
y=df.iuu_caught

### One Hot Encoding

In [5]:
print(df['vessel_class'].unique())
print(df['Ocean List New whose false were in OG'].unique())

['other' 'trawlers' 'squid_jigger' 'tuna_purse_seines'
 'drifting_longlines']
['atlantic' 'southern' 'mediterranean' 'arctic' 'pacific' 'Pacific'
 'FALSE' 'indian']


In [6]:
print(df['vessel_class'].value_counts())
print(df['Ocean List New whose false were in OG'].value_counts())

drifting_longlines    18641
squid_jigger          16021
tuna_purse_seines      8620
trawlers               7913
other                  4173
Name: vessel_class, dtype: int64
pacific          17108
atlantic         16469
Pacific          14762
indian            6176
arctic             499
FALSE              227
southern           116
mediterranean       11
Name: Ocean List New whose false were in OG, dtype: int64


In [7]:
one_hot_encoded_data = pd.get_dummies(df, columns = ['vessel_class', 'Ocean List New whose false were in OG'])
print(one_hot_encoded_data.head())

   gap_hours  iuu_caught  spherical_distances  eez_check  \
0  13.016667           0           207.942845          0   
1  13.850000           0             9.704232          0   
2  33.733333           0            10.789069          1   
3  30.650000           0            11.695860          1   
4  50.283333           0            93.113335          1   

   vessel_class_drifting_longlines  vessel_class_other  \
0                                0                   1   
1                                0                   1   
2                                0                   1   
3                                0                   1   
4                                0                   1   

   vessel_class_squid_jigger  vessel_class_trawlers  \
0                          0                      0   
1                          0                      0   
2                          0                      0   
3                          0                      0   
4              

In [8]:
#### Independent and Dependent Features
X=one_hot_encoded_data.drop("iuu_caught",axis=1)
y=one_hot_encoded_data.iuu_caught

#### SMOTETomek

In [9]:
y.value_counts()

0    55268
1      100
Name: iuu_caught, dtype: int64

In [10]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X,y)

y_sm.value_counts()

0    55268
1    55268
Name: iuu_caught, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=15, stratify=y_sm)

In [12]:
y_train.value_counts()

0    44214
1    44214
Name: iuu_caught, dtype: int64

In [13]:
y_test.value_counts()

0    11054
1    11054
Name: iuu_caught, dtype: int64

In [14]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report
# from tensorflow_addons import losses


def ANN(X_train, y_train, X_test, y_test, loss, weights):
    model = keras.Sequential([
        keras.layers.Dense(26, input_dim=16, activation='relu'),
        keras.layers.Dense(15, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    
    if weights == -1:
        model.fit(X_train, y_train, epochs=50)
    else:
        model.fit(X_train, y_train, epochs=10, class_weight = weights)
    
    print(model.evaluate(X_test, y_test))
    
    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)
    
    print("Classification Report: \n", classification_report(y_test, y_preds))
    
    return y_preds

y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/50
2764/2764 [==============================] - 10s 3ms/step - loss: 1.1827 - accuracy: 0.7951
Epoch 2/50
2764/2764 [==============================] - 7s 3ms/step - loss: 0.3273 - accuracy: 0.8949
Epoch 3/50
2764/2764 [==============================] - 7s 3ms/step - loss: 0.2867 - accuracy: 0.9126
Epoch 4/50
2764/2764 [==============================] - 7s 3ms/step - loss: 0.2379 - accuracy: 0.9219
Epoch 5/50
2764/2764 [==============================] - 7s 3ms/step - loss: 0.2470 - accuracy: 0.9212
Epoch 6/50
2764/2764 [==============================] - 7s 3ms/step - loss: 0.2284 - accuracy: 0.9241
Epoch 7/50
2764/2764 [==============================] - 7s 3ms/step - loss: 0.2016 - accuracy: 0.9285
Epoch 8/50
2764/2764 [==============================] - 8s 3ms/step - loss: 0.2047 - accuracy: 0.9306
Epoch 9/50
2764/2764 [==============================] - 7s 3ms/step - loss: 0.1847 - accuracy: 0.9363
Epoch 10/50
2764/2764 [==============================] - 7s 3ms/step - loss: 0.16